We define a generic class InputGeneric which is inherited to InputVariable and InputVariablesGroup.
We detect a variable (InputVariable) by the presence of the key 'default' otherwise it is a group of variables (inputVariablesGroup).

In [29]:
from difflib import SequenceMatcher
import yaml

class InputGeneric():
    """Generic class to define input variables or group of input variables."""
    def __init__(self,name,definition,parent=None):
        """From the dictionary import the tree of variables."""
        self.name = name
        self.parent = parent
        self.__rawdict__ = definition.copy()
        self.DESCRIPTION = self.__rawdict__.pop('DESCRIPTION','')
        self.__vars__ = self.__rawdict__.keys()
        self.__vars__.sort()
        #By default check the last level of input variables group.
        self.__safeload = False
    def __str__(self):
        """Return the full name of the variable with its parents"""
        if self.parent:
            return str(self.parent)+"."+self.name
        else:
            return self.name
    def full(self):
        """Return a full input yaml document"""
        return yaml.dump(self.todict(),default_flow_style=False)
    def load(self,string,safe=True):
        """Load a yaml dict and set the corresponding variables.
           If the option 'safe' is false, do not check if an input variable does exist."""
        if isinstance(string,str) or isinstance(string,file):
            doc = yaml.load(string)
        elif isinstance(string,dict):
            doc = string
        self.load_doc(doc,safe)
    def load_doc(self,doc,safe=True):
        """Load a yaml dict and set the corresponding variables.
           If the option 'safe' is false, do not check if an input variable does exist."""
        for (key,val) in doc.items():
            if key in self.__vars__:
                var = getattr(self,key)
                var.load_doc(val)
            elif safe and self.__safeload:
                raise KeyError("The key '%s' for '%s' does not exist!" % (key,self.name))
    def match(self,searchfor):
        """Compare a sequence to all keys"""
        matching_keys = []
        for k in self.__vars__:
            var = getattr(self,k)
            m = var.match(searchfor)
            if (isinstance(m,list)):
                matching_keys.extend(m)
            else:
                matching_keys.append(m)
        matching_keys.sort(key=lambda x: x[1])
        matching_keys.reverse()
        return matching_keys
    def minimal(self):
        """Return a minimal input yaml document"""
        return yaml.dump(self.tomin(),default_flow_style=False)
    def todict(self):
        """Translate into a dictionary"""
        return {k: getattr(self,k).todict() for k in self.__vars__}
    def tomin(self):
        """Return a minimal input yaml document"""
        doc = {k: getattr(self,k).tomin() for k in self.__vars__}
        for (key,val) in doc.items():
            if val == 'default' or val == {}:
                del doc[key]
        return doc
    def wiki(self):
        """Wiki representation of the input variables"""
        cr="\r\n"
        string="="+self.name+"="+cr
        for var in self.__vars__:
            variable=getattr(self,var)
            string+=wiki(variable)
        return string
    
class InputVariablesGroup(InputGeneric):
    """Define group of input variables."""
    def __init__(self,name,definition,parent=None):
        InputGeneric.__init__(self,name,definition,parent)
        for var in self.__rawdict__:
            if definition[var].has_key('default'):
                setattr(self,var,InputVariable(var,definition[var],self))
                #Last level of groups of input variables
                self.__safeload = True
            else:
                setattr(self,var,InputVariablesGroup(var,definition[var],self))
    
class InputVariable(InputGeneric):
    """Class of input variable"""
    def __init__(self,name,definition,parent=None):
        """Define the variable associated to the input variable in futile proposed format"""
        #Raw definiton
        InputGeneric.__init__(self,name,definition,parent)
        #Set the attribute and the meta values
        self.__meta = []
        for key in definition:
            setattr(self,key,definition[key])
            if key not in ['COMMENT', 'DESCRIPTION', 'default', 'CONDITION', 'EXCLUSIVE', 'RANGE']:
                self.__meta.append(key)
        #Set the default value
        self.set(self.default)
    def __str__(self):
        """return the full name and the value of the variable"""
        return InputGeneric.__str__(self)+": "+str(self.value)
    def load_doc(self,doc):
        """Set the value from a document"""
        self.set(doc)
    def match(self,searchfor):
        """Give the match of the input variable with a given sequence"""
        #s = SequenceMatcher(lambda x: x == " ", searchfor, 
        #                    "%s %s %s" % (self.name,self.COMMENT,self.DESCRIPTION))
        slow=searchfor.lower()
        c = SequenceMatcher(lambda x: x == " ", slow,self.COMMENT.lower())
        print self.COMMENT,c.ratio()*len(slow)
        d = SequenceMatcher(lambda x: x == " ", slow,self.DESCRIPTION.lower())
        print self.DESCRIPTION,d.ratio()
        n = SequenceMatcher(lambda x: x == " ", slow,self.name.lower())
        print self.name,n.ratio()
        return (self, n.ratio()+c.ratio()+d.ratio())
    def set(self,value):
        """Set the value for the corresponding variables and check it."""
        #Special case for default
        if value == 'default': 
            self.value = self.default
        else:
            self.value = value
        #Check if the value is equal to a meta value and replace by the meta key
        for m in self.__meta:
            if self.value == getattr(self,m):
                self.value = m
        if self.value in self.__meta:
            pass
        #here we can check that the value is legal following the definitions
        elif hasattr(self,'EXCLUSIVE') and self.value not in self.EXCLUSIVE:
            raise TypeError('Permitted value: '+str(self.EXCLUSIVE)+ \
                            "\nValue ('"+str(self.value)+"') not admitted for variable "+str(self.name))
        elif hasattr(self,'RANGE') and (self.value < self.RANGE[0] or self.value > self.RANGE[1]):
            raise TypeError('Permitted range: '+str(self.RANGE)+ \
                            "\nValue ("+str(self.value)+") not admitted for variable "+str(self.name))
    def todict(self):
        """Return the value"""
        return self.value
    def tomin(self):
        """Return the value if it is not the default"""
        if self.value == self.default:
            return 'default'
        else:
            return self.value
    def wiki(self):
        cr="\r\n"
        string="=="+self.name+"=="+cr
        if hasattr(self,'DESCRIPTION'):
            string+=self.DESCRIPTION+cr
        elif hasattr(self,'COMMENT'):
            string+=self.COMMENT+cr
        string+="Default value: "+str(self.default)+cr
        if hasattr(self,'EXCLUSIVE'):
            for ex in self.EXCLUSIVE:
                string+='* '+str(ex)+cr
        return string

def closest_keys(inputs,searchfor):
    """Find the closest input variables from searchfor"""
    doc = []
    for (var,ratio) in inputs.match(searchfor):
        if ratio >= 0.5:
            doc.append(var)
    return doc
    #print yaml.dump(doc, default_flow_style=False, explicit_start=True)
    
class FindKey(list):
    #def __new__(cls,:
    #    return list.__new__(cls) 
    def __init__(self,inputs,searchfor):
        self += closest_keys(inputs,searchfor)
        self.searchfor=searchfor
    def __str__(self):
        string='Keys which match the most with "%s"\n' % self.searchfor
        string+=str([ (c.name,c.COMMENT) for c in self])
        return string

In [10]:
import yaml
isf=yaml.load('''
  isf_order: 
    COMMENT: Order of the Interpolating Scaling Function family
    DESCRIPTION: Fixes the order of the ISF family that is used for the discretization of the kernel
    default: 16
    EXCLUSIVE: [ 2, 4, 6, 8, 14, 16, 20, 24, 30, 40, 50, 60, 100]
''')
print isf

isfvar=InputVariable('isf_order',isf['isf_order'])
print isfvar.wiki()
print isfvar.todict()
a = isfvar.match('isf')
print a[0].name,a[1]

{'isf_order': {'COMMENT': 'Order of the Interpolating Scaling Function family', 'default': 16, 'DESCRIPTION': 'Fixes the order of the ISF family that is used for the discretization of the kernel', 'EXCLUSIVE': [2, 4, 6, 8, 14, 16, 20, 24, 30, 40, 50, 60, 100]}}
==isf_order==
Fixes the order of the ISF family that is used for the discretization of the kernel
Default value: 16
* 2
* 4
* 6
* 8
* 14
* 16
* 20
* 24
* 30
* 40
* 50
* 60
* 100

16
isf_order 0.0408163265306


In [30]:
docs=[a for a in yaml.load_all(open('../../psolver/src/PS_input_variables_definition.yaml','r'))]
ps=docs[0]

environment = InputVariablesGroup("environment",ps["environment"])
cc=closest_keys(environment,'min')
print cc,repr(cc),str(cc) # [ (c.name, c.COMMENT) for c in cc]

cc= FindKey(environment,'dielectric')
print cc


psolver = InputVariablesGroup('psolver',ps)
psolver.environment.epsilon.set(78.36)

print psolver.environment.epsilon.value == 78.36
print psolver.environment.epsilon.value == 'water'

#doc = yaml.load(open('test-psolver.yaml','r'))
psolver.load(open('test-psolver.yaml','r'))
#psolver.load(doc)

print psolver.minimal()

Proportionality of repulsion free energy in term of the surface integral [dyn/cm] 0.142857142857
 0.0
alphaS 0.0
Mapping of the radii that have to be used for each atomic species 0.176470588235
 0.0
atomic_radii 0.266666666667
Proportionality of dispersion free energy in term of volume integral [GPa] 0.155844155844
 0.0
betaV 0.0
Triggers the evaluation of the extra cavitation terms 0.107142857143
 0.0
cavitation 0.307692307692
Type of the cavity 0.285714285714
When not none, a cavity around the density is fixed 0.0740740740741
cavity 0.222222222222
Amplitude of the transition region in the rigid cavity (AU) 0.290322580645
 0.0
delta 0.0
Extremum values of the charge density, for the definition of the transition region 0.211764705882
 0.0
edensmaxmin 0.428571428571
Dielectric constant of the exterior region 0.266666666667
 0.0
epsilon 0.4
Multiplying factor for the whole rigid cavity 0.375
 0.0
fact_rigid 0.153846153846
Order of the Finite-difference derivatives for the GPS solver 0.18

In [4]:
print psolver.environment.alphaS

psolver.environment.alphaS: water


In [5]:
psolver.environment.load("dft: OK")


In [6]:
print psolver.environment.dft


AttributeError: InputVariablesGroup instance has no attribute 'dft'

In [8]:
docs=[a for a in yaml.load_all(open('../../bigdft/src/input_variables_definition.yaml','r'))]
bigdft=InputVariablesGroup('bigdft',docs[0])

TypeError: Permitted range: [0, 99999.0]
Value (20.d0) not admitted for variable timestep